In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment, Workspace
from azureml.core.runconfig import DockerConfiguration
from azureml.core.compute import ComputeTarget
from google.cloud import secretmanager
import wandb

# Connect to Azure ML Workspace
workspace = Workspace(subscription_id="7ec8e6b8-bef0-4811-91a9-6bf389d80de2", resource_group="rg-llmopsaccel-01dev", workspace_name="mlw-llmopsaccel-01dev")

# Ensure Compute Target Exists
compute_target = ComputeTarget(workspace=workspace, name="gpu-cluster-sleep")

# Define Azure ML Environment (With W&B API Key)
env = Environment.from_pip_requirements(
    name="topic-assignment-env",
    file_path="azure_requirements.txt"
)
env.docker.base_image = "mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.6-cudnn8-ubuntu20.04"
docker_config = DockerConfiguration(use_docker=True)


def get_secret(project_id: str, secret_id: str, version_id: str = "latest") -> str:
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")

# Usage
project = "923326131319"
secret  = "WANDB_API_KEY_DAVIDE"
wandb_api_key = get_secret(project, secret)

env.environment_variables = {
    "WANDB_API_KEY": wandb_api_key
}

# ✅ Register the environment in Azure ML
env.register(workspace=workspace)

print("Environment registered successfully!")

Property environment_variables is deprecated. Use RunConfiguration.environment_variables to set runtime variables.


Environment registered successfully!


In [ ]:
# Configure training script submission
script_config = ScriptRunConfig(
    source_directory=".",
    script="deBERTa-pairwise.py",
    compute_target=compute_target,
    environment=env,
    docker_runtime_config=docker_config
)

# Submit the experiment to Azure ML
experiment = Experiment(workspace, "topic-assignment")
run = experiment.submit(script_config)
run.wait_for_completion(show_output=True)

# Print logs
logs = run.get_details()
print(logs)

RunId: topic-assignment_1745936704_3793f36a
Web View: https://ml.azure.com/runs/topic-assignment_1745936704_3793f36a?wsid=/subscriptions/7ec8e6b8-bef0-4811-91a9-6bf389d80de2/resourcegroups/rg-llmopsaccel-01dev/workspaces/mlw-llmopsaccel-01dev&tid=720b637a-655a-40cf-816a-f22f40755c2c

Streaming user_logs/std_log.txt

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: No netrc file found, creating one.
wandb: Appending key for wandb.mlops.ingka.com to your netrc file: /root/.netrc
wandb: Currently logged in as: davide-zanutto (digital-ethics-responsible-ai) to https://wandb.mlops.ingka.com. Use `wandb login --relogin` to force relogin
Create sweep with ID: avxxnbah
Sweep URL: https://wandb.mlops.ingka.com/digital-ethics-responsible-ai/topic-assignment/sweeps/avxxnbah
wandb: Agent Starting Run: keuawvwv with config:
wandb: 	learning_rate: 2.9849271543140006e-05
wandb: 	num_train_epochs: 5
wandb: 	per_device_train_batch_siz

/Users/davide.zanutto1/Desktop/ratings-reviews-ai-summaries-topics/ratings-reviews/lib/python3.12/site-packages/azureml/_base_sdk_common/common.py:102: SyntaxWarning: invalid escape sequence '\w'
  return re.match("^[a-zA-Z0-9][\w\-]{2,32}$", workspace_name)
/Users/davide.zanutto1/Desktop/ratings-reviews-ai-summaries-topics/ratings-reviews/lib/python3.12/site-packages/azureml/_base_sdk_common/common.py:109: SyntaxWarning: invalid escape sequence '\w'
  regex = "^[a-zA-Z0-9][\w\-]{{{},{}}}$".format(EXPERIMENT_LENGTH_MIN-1, EXPERIMENT_LENGTH_MAX-1)


AuthenticationException: AuthenticationException:
	Message: Please ensure you have network connection. Error detail: HTTPSConnectionPool(host='login.microsoftonline.com', port=443): Max retries exceeded with url: /organizations/v2.0/.well-known/openid-configuration (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x11ae78dd0>: Failed to resolve 'login.microsoftonline.com' ([Errno 8] nodename nor servname provided, or not known)"))
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "inner_error": {
            "code": "Authentication"
        },
        "message": "Please ensure you have network connection. Error detail: HTTPSConnectionPool(host='login.microsoftonline.com', port=443): Max retries exceeded with url: /organizations/v2.0/.well-known/openid-configuration (Caused by NameResolutionError(\"<urllib3.connection.HTTPSConnection object at 0x11ae78dd0>: Failed to resolve 'login.microsoftonline.com' ([Errno 8] node